# Wprowadzenie 
Nasze wyzwanie jest z jednej strony proste, z drugiej strony dość ambitne. 

Jedno z klasycznych "Hello World" świata Big Data polega na zliczaniu wystąpienia słów. Dane wejściowe - plik tekstowy lub strumień tekstu. Dane wynikowe - liczba wystąpień każdego ze słów. Klasyka. 

My zrobimy to samo, jednak naszymi danymi wejściowymi będą... opowiadania Artura Conan Doyla (czyli standard), ale nie w plikach tekstowych, a w formacie PDF (i to już standard nie jest). 

Trudne? Nic bardziej mylnego. Python to mnogość bibliotek o niezliczonej funkcjonalności. 

Prosty przykład...

Pobierz nasze dane wejściowe

In [ ]:
import requests
r = requests.get("https://jankiewicz.pl/bigdata/bigdata-sp/cano-pdf.zip", allow_redirects=True)
open('cano-pdf.zip', 'wb').write(r.content)

Rozpakuj nasz plik

In [ ]:
%%sh
unzip cano-pdf.zip

Sprawdź czy mamy zainstalowany potrzebny moduł

# PyPDF2

In [ ]:
%%sh
pip freeze | grep PyPDF2

In [ ]:
import PyPDF2 
    
# Utwórz obiekt odnoszący się do przykładowego pliku
pdfFileObj = open('cano-pdf/3gab.pdf', 'rb') 
    
# Utwórz obiekt PdfFileReader 
pdfReader = PyPDF2.PdfReader(pdfFileObj) 
    
# To wszystko 
# Zobacz ile ten plik ma stron 
print(len(pdfReader.pages))

In [ ]:
# Pobierz pierwszą ze stron
pageObj = pdfReader.pages[0]
    
# Dokonaj esktrakcji tekstu, który się na niej znajduje 
print(pageObj.extract_text()) 

In [ ]:
# Nie zapomnij zamknąć nasz obiekt pliku
pdfFileObj.close() 

Proste prawda? 

No to do roboty. W pierwszej kolejności załadujmy dane tam, gdzie będą one mogły być wydajnie odczytywane przez wiele węzłów klastra

# Przygotowanie danych

In [ ]:
%%sh
hadoop fs -mkdir -p cano-pdf

In [ ]:
%%sh
hadoop fs -put -f cano-pdf/* cano-pdf/

In [ ]:
%%sh
hadoop fs -ls cano-pdf

Utwórzmy teraz nasz obiekt konteksu (o ile jeszcze nie istnieje)

# Utworzenie obiektu kontekstu

In [ ]:
# w przypadku korzystania z kernela Python
from pyspark import SparkContext, SparkConf

In [ ]:
# w przypadku korzystania z kernela Python
conf = SparkConf().setAppName("Spark - RDD - warsztaty").setMaster("yarn")
sc = SparkContext(conf=conf)

Do tej pory szło gładko. Teraz mamy mały problem. <br> 
W jaki sposób zaczytać nasze pliki? 

Nie są to pliki tekstowe, więc `textFile` prowadzający dane linia po linii do naszych dokumentów nie jest tu przydatny.<br>
Zaglądnij na https://spark.apache.org/docs/latest/rdd-programming-guide.html#external-datasets

Właściwie, żadna z metod nie jest tu odpowiednia. 

Zrobimy zatem tak, naszymi danymi wejściowymi nie będą pliki. Będą ich nazwy, a Spark na podstawie tych nazw będzie je odczytywał i ... 

# Przygotowanie metadanych wejściowych

In [ ]:
sc

In [ ]:
%%sh
hadoop fs -ls cano-pdf > files.txt

In [ ]:
%%sh
hadoop fs -copyFromLocal files.txt

In [ ]:
rawFiles = sc.textFile("files.txt")

In [ ]:
rawFiles.collect()

Jesteśmy zainteresowani tylko nazwami plików, a zatem...

In [ ]:
import re
rawFiles.filter(lambda s: "cano" in s).map(lambda s: re.search(".* (\S*)$",s).group(1)).collect()

In [ ]:
fileNames = rawFiles.filter(lambda s: "cano" in s).map(lambda s: re.search(".* (\S*)$",s).group(1))

Nie chcemy aby całą ekstrakcję danych tekstowych z plików PDF wykonywał jeden węzeł. Sprawdźmy ile mamy partycji naszego RDD. 

Jeśli będzie ich zbyt mało, możemy zmienić ich liczbę za pomoca metody `repartition(liczba_partycji)`.

Przeanalizuj to ile zasobów ma nasz klaster, w szczególności zwróć uwagę na liczbę procesorów we wszystkich maszynach.

Stosując *regułę kciuka* ustaw liczbę partycji na taką, która jest równa liczbie procesorów. Wprowadź zmiany w powyższej linii, tak aby poniższa potwierdziła oczekiwaną liczbę partycji. 

In [ ]:
fileNames.getNumPartitions()

# Konwersja metadanych na dane 

Jeśli liczba partycji jest już w porządku, to czas na kluczowy moment. <br>
Chcemy, aby każdy z elementów naszego RDD zamienił się z nazwy pliku, na szereg elementów odnoszących się do poszczególnych linii zawartych w tym pliku. 

Potrzebujemy zatem funkcji, która:
* odczyta plik o podanej nazwie 
* dokona ekstracji jego zawartości
* utworzy listę zawierającą poszczególne linie

Funkcję tą wykorzystamy następnie w metodzie `flatMap` na naszym `RDD`. <br>
Reszta będzie *easy peasy*. 

**Uwaga!** <br>
Plik nie będzie znajdował się w lokalnym systemie plików węzła roboczego... będzie znajdował się w systemie plików HDFS!

Aby sobie z tym poradzić, sprawdźmy czy mamy dostępną jeszcze jedną bibliotekę.

In [ ]:
%%sh
pip freeze | grep pydoop

In [ ]:
def pdf2txt(fileName):
    
    import PyPDF2
    import pydoop.hdfs as hdfs

    # Utwórz obiekt odnoszący się do przykładowego pliku
    pdfFileObj = hdfs.open(fileName, "rb") 
    
    # Utwórz obiekt PdfFileReader 
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj) 
    
    lines = []
    
    for page in range(len(pdfReader.pages)): 
        pageObj = pdfReader.pages[page] 
        content = pageObj.extract_text() 
        lines.extend(content.splitlines())
    pdfFileObj.close()
    
    return lines

Sprawdźmy ją. Tym razem będzie to odczyt z systemu plików HDFS.

In [ ]:
lines_3gab = pdf2txt("cano-pdf/3gab.pdf")

In [ ]:
lines_3gab[:3]

Pozostało nam z niej skorzystać.

In [ ]:
lines = fileNames.flatMap(lambda fn: pdf2txt(fn))

Próba generalna

In [ ]:
lines.take(2)

# Zadania 

Teraz już z górki. Reszta należy do Ciebie. 

**Uwaga!** Na wynikowym RDD, który powinien zawierać dla każdego słowa liczbę jego wystąpień, będziemy wykonywali wiele operacji. <br>
Zadbaj o to, aby każdorazowe użycie tego wynikowego RDD nie powodowało odczytywania plików PDF.

## Zadanie 1

Utwórz obiekt RDD `wordCounts`, który dla każdego słowa liczbę jego wystąpień.

In [ ]:
import re
words = lines

In [ ]:
wordCounts = words

## Zadanie 2

Znajdź 10 najczęściej wykorzystywanych słów.

In [ ]:
wordCounts

## Zadanie 3

Znajdź 10 najczęściej wykorzystywanych słów, które składają się z co najmniej 5 liter. 

In [ ]:
wordCounts

## Zadanie 4

Ile razy pojawiło się słowo "Watson"?

In [ ]:
wordCounts

## Zadanie 5

A ile razy pojawiło się słowo "Moriarty"?

In [ ]:
wordCounts